In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#train and test
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
train.shape, test.shape

In [3]:
#importing libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
train.head()

In [5]:
features = [x for x in train.columns.values if x[0]=="f"]

In [6]:
train['is_na'] = train[features].isna().sum(axis=1)
train['abs'] = train[features].abs().sum(axis=1)
train['sem'] = train[features].sem(axis=1)
train['std'] = train[features].std(axis=1)
train['avg'] = train[features].mean(axis=1)
train['max'] = train[features].max(axis=1)
train['min'] = train[features].min(axis=1)

test['is_na'] = test[features].isna().sum(axis=1)
test['abs'] = test[features].abs().sum(axis=1)
test['sem'] = test[features].sem(axis=1)
test['std'] = test[features].std(axis=1)
test['avg'] = test[features].mean(axis=1)
test['max'] = test[features].max(axis=1)
test['min'] = test[features].min(axis=1)

In [7]:
for i in features:
    print(i, train[i].skew())

In [8]:
for i in features:
    sns.displot(train[i])

In [9]:
mean_fea = ['f1','f6','f15','f34','f36','f57','f79','f113']
for i in mean_fea:
    train[i].fillna(train[i].mean(),inplace=True)
    test[i].fillna(test[i].mean(),inplace=True)

In [10]:
train.head()

In [11]:
medi = train.drop(mean_fea, axis=1)
medi_fea = medi.drop(['id','claim'], axis=1)
medi_fea_final = medi_fea.columns
medi_fea_final

In [12]:
for i in medi_fea_final:
    train[i].fillna(train[i].median(), inplace= True)
    test[i].fillna(test[i].median(), inplace= True)

In [13]:
train.isna().sum()

In [14]:
test.isna().sum()

In [15]:
test.head()
test = test.drop('id',axis=1)

In [16]:
y = train['claim']
x = train.drop(['id','claim'],axis=1)

In [17]:
test.head()

In [18]:
x.head()

In [19]:
x.shape, test.shape

In [20]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()

x = rs.fit_transform(x)
test = rs.transform(test)

In [21]:
from sklearn.model_selection import StratifiedKFold
splits = 10
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for fold, (idx_train, idx_valid) in enumerate(kf.split(x, y)):
    # create train, validation sets
    X_train, y_train = x[idx_train], y[idx_train]
    X_valid, y_valid = x[idx_valid], y[idx_valid]

In [22]:
from lightgbm import LGBMClassifier
from sklearn import metrics
model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred_final = model.predict_proba(X_valid)[:,1]
auc = metrics.roc_auc_score(y_valid, y_pred_final)
auc

In [23]:
test_dummy = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
test_id = test_dummy['id']

In [24]:
y_prdict =model.predict_proba(test)[:,1]

In [25]:
y_prdict

In [26]:
submission = pd.DataFrame(list(zip(test_id, y_prdict)),
               columns =['id', 'claim'])
submission.head(20)

In [27]:
submission.to_csv("Submission.csv",index=False)